In [65]:
from CNN_model_v3 import * 
from CNN_RNN_v3 import * 
from CNN_RNN_att_v3 import * 
from raim_v3 import *
from raim_model_w_att import * 

#  1. Train CNN on short-term (hourly) ECG signals


In [2]:
model = cnn_model()
X = np.random.randn(10, 450000, 1)
Y = np.ones((10))
Y[:5]=0
values = Y

integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Y = onehot_encoded

x_tr, x_te, y_tr, y_te = train_test_split(X, Y, test_size=0.1, random_state=0)

filepath="./m_weights-CNN_DC-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


# if the accuracy does not increase over 10 epochs, we reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
metrics_history = MetricsHistory()


model.fit(x= x_tr,
          y= y_tr,
          batch_size=1,
          epochs=1,
          verbose=1,
          shuffle=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


/Users/siddharthbiswal/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch 1/1
9/9 [==============================] - 11s - loss: 0.9249 - acc: 0.2222    


# 2. Train CNN-RNN on long-term (e.g., 12-hr) sequences¶


In [66]:
model = get_model_lstm_w_att()

file_path = "lstm_model.h5"


checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=20, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=5, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

base_path = "./test_data/"

files = glob.glob((os.path.join(base_path, "*.npz")))

ids = sorted(list(set([x.split("/")[-1][:6] for x in files])))
#split by test subject
train_ids, test_ids = train_test_split(ids, test_size=0.15, random_state=1338)

train_val, test = [x for x in files if x.split("/")[-1][:6] in train_ids],\
                  [x for x in files if x.split("/")[-1][:6] in test_ids]

train, val = train_test_split(train_val, test_size=0.1, random_state=1337)

train_dict = {k: np.load(k) for k in train}
test_dict = {k: np.load(k) for k in test}
val_dict = {k: np.load(k) for k in val}


model.fit_generator(gen(train_dict, aug=False), validation_data=gen(val_dict), epochs=2, verbose=2,
                    steps_per_epoch=10, validation_steps=10, callbacks=callbacks_list)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, None, 3750, 1)     0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 64)          250416    
_________________________________________________________________
bidirectional_9 (Bidirection (None, None, 200)         132000    
_________________________________________________________________
dropout_15 (Dropout)         (None, None, 200)         0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, None, 200)         240800    
_________________________________________________________________
conv1d_47 (Conv1D)           (None, None, 2)           402       
Total params: 623,618
Trainable params: 623,618
Non-trainable params: 0
_________________________________________________________________
Epoc

# 3. Train CNN-AttRNN on long-term sequences¶


In [68]:
model = get_model_lstm()

file_path = "lstm_model.h5"


checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=20, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=5, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

base_path = "./test_data/"

files = glob.glob((os.path.join(base_path, "*.npz")))

ids = sorted(list(set([x.split("/")[-1][:6] for x in files])))
#split by test subject
train_ids, test_ids = train_test_split(ids, test_size=0.15, random_state=1338)

train_val, test = [x for x in files if x.split("/")[-1][:6] in train_ids],\
                  [x for x in files if x.split("/")[-1][:6] in test_ids]

train, val = train_test_split(train_val, test_size=0.1, random_state=1337)

train_dict = {k: np.load(k) for k in train}
test_dict = {k: np.load(k) for k in test}
val_dict = {k: np.load(k) for k in val}


model.fit_generator(gen(train_dict, aug=False), validation_data=gen(val_dict), epochs=2, verbose=2,
                    steps_per_epoch=10, validation_steps=10, callbacks=callbacks_list)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, None, 3750, 1) 0                                            
____________________________________________________________________________________________________
time_distributed_11 (TimeDistrib (None, None, 64)      250416      input_13[0][0]                   
____________________________________________________________________________________________________
bidirectional_13 (Bidirectional) (None, None, 200)     132000      time_distributed_11[0][0]        
____________________________________________________________________________________________________
dropout_21 (Dropout)             (None, None, 200)     0           bidirectional_13[0][0]           
___________________________________________________________________________________________

# 4. Train RAIM with lab-guided attention¶


In [71]:
# RAIM definition
# model definition -- use full attention when there is no guidance 
class RNNNetv1(nn.Module):
    def __init__(self, batch_size, hidden_size):
        super(RNNNetv1, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.rnn = nn.LSTM(500, 200, 2, dropout=0.5)
        self.attn = nn.Linear(10, 10)
        self.attn1 = nn.Linear(60,10)

        self.dense_h = nn.Linear(200,1)
        self.softmax = nn.Softmax()
        

        self.hidden2label = nn.Linear(200, 2)
        self.hidden = self.init_hidden()
        self.grucell = nn.GRUCell(500, 200)
        
        self.mlp_for_x      = nn.Linear(500,1, bias=False)
        self.mlp_for_hidden = nn.Linear(200,12,bias=True)
        

    def init_hidden(self):
        return Variable(torch.zeros(self.batch_size, self.hidden_size))


    def forward(self, x, guidance):        
        for i in range(12):            
            tt = x[:,0:i+1,:].reshape(self.batch_size, (i+1)* x[:,0:i+1,:].shape[2])
            if i<11:
                padding = torch.zeros(self.batch_size, 6000-tt.shape[1])
                self.temp1 = torch.cat((tt, padding),1)
            else:
                self.temp1 = tt
           
            self.input_padded = self.temp1.reshape(10,12,500)
            
            #### multuply with guidance #######
            temp_guidance = torch.zeros(10,12,1)
            
            temp_guidance[:,0:i+1,:] = guidance[:,0:i+1,:]
          
            if i>0:
               
                zero_idx = np.where(torch.sum(guidance[:,:i,0], dim=1)==0)
                if len(zero_idx[0])>0:

                    temp_guidance[zero_idx[0],:i,0] = 1

            temp_guidance[:,i,:] = 1

            self.guided_input = torch.mul(self.input_padded, temp_guidance)
            
            ######### MLP ###########
            self.t1 = self.mlp_for_x(self.guided_input) + self.mlp_for_hidden(self.hidden).reshape(10,12,1)
            
            ######### softmax-> multiply->  context vector ###########
            self.t1_softmax = self.softmax(self.t1)
            final_output = torch.mul(self.input_padded, self.t1_softmax)

            context_vec = torch.sum(final_output,dim=1)            
            
            self.hx = self.grucell(context_vec, self.hidden)
            self.hidden = self.hx
            
        y  = self.hidden2label(self.hidden)
        return self.hidden

In [73]:

x_tr = np.load('X_input_RAIM.npy')
y_target = np.load('Y_target.npy')
y_tr = np.load('Y_guidance.npy')

trainset_v1 = TensorDataset(torch.from_numpy(x_tr.astype('float32')), torch.from_numpy(y_tr.astype('float32')),torch.from_numpy(y_guidance.astype('float32')))
trainloader_v1 = torch.utils.data.DataLoader(trainset_v1, batch_size=32, shuffle=True, num_workers=2)

dataiter = iter(trainloader_v1)
X_samples = dataiter.next()

trainset_v1 = TensorDataset(torch.from_numpy(x_tr.astype('float32')), torch.from_numpy(y_tr.astype('float32')),torch.from_numpy(y_guidance.astype('float32')))
trainloader_v1 = torch.utils.data.DataLoader(trainset_v1, batch_size=32, shuffle=True, num_workers=2)

dataiter = iter(trainloader_v1)

print('input shape:', X_samples[0].shape)

model = RNNNetv1(hidden_size = 200, batch_size = 10)
output = model(X_samples[0], X_samples[2])
print(output.shape)


('input shape:', torch.Size([10, 12, 500]))
torch.Size([10, 200])


/Users/siddharthbiswal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# 5. Train RAIM with medication-guided attention¶


In [74]:

x_tr = np.load('X_input_RAIM.npy')
y_target = np.load('Y_target.npy')
y_tr = np.load('Y_guidance_med.npy')

trainset_v1 = TensorDataset(torch.from_numpy(x_tr.astype('float32')), torch.from_numpy(y_tr.astype('float32')),torch.from_numpy(y_target.astype('float32')))
trainloader_v1 = torch.utils.data.DataLoader(trainset_v1, batch_size=32, shuffle=True, num_workers=2)

dataiter = iter(trainloader_v1)
X_samples = dataiter.next()

print('input shape:', X_samples[0].shape)

model = RNNNetv1(hidden_size = 200, batch_size = 10)
output = model(X_samples[0], X_samples[2])
print(output.shape)



('input shape:', torch.Size([10, 12, 500]))
torch.Size([10, 200])


/Users/siddharthbiswal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# 6. Train RAIM with both guidence

In [62]:

x_tr = np.load('X_input_RAIM.npy')
y_target = np.load('Y_target.npy')
y_tr = np.load('Y_guidance_combined.npy')

trainset_v1 = TensorDataset(torch.from_numpy(x_tr.astype('float32')), torch.from_numpy(y_tr.astype('float32')),torch.from_numpy(y_target.astype('float32')))
trainloader_v1 = torch.utils.data.DataLoader(trainset_v1, batch_size=32, shuffle=True, num_workers=2)

dataiter = iter(trainloader_v1)
X_samples = dataiter.next()


model = RNNNetv1(hidden_size = 200, batch_size = 10)
output = model(X_samples[0], X_samples[2])
print(output.shape)



torch.Size([10, 200])


/Users/siddharthbiswal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
